In [ ]:
# Editor appearance set up & Load image

# Extend width of Jupyter Notebook Cell to the size of browser
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


# Import packages needed
import gc
import pickle
import platform
from tkinter import Tk
from tkinter.filedialog import askopenfilename, asksaveasfilename

from PIL import Image
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
from matplotlib.widgets import RectangleSelector, PolygonSelector
import numpy as np
from skimage import io, draw
import mplcursors
import IPython.display as Disp
from ipywidgets import widgets
import cv2
from osgeo import gdal


# OS related settings
if platform.system() == 'Windows':
    print('Windows')
    # %matplotlib nbagg
    # Sometimes tk/qt will not let cells rerun after an ERROR occurs
#     %matplotlib tk
    %matplotlib qt
elif platform.system() == 'Darwin':
    print('macOS')
    Tk().withdraw()
    %matplotlib osx
elif platform == 'linux' or platform == 'linux2':
    print('Linux')
# This line of "print" must exist right after %matplotlib command, 
# otherwise JN will hang on the first import statement after this.
print('Interactive plot activated')


# Load image and print size & pre-process

# Use skimage to load multi-layer tiff file
# Tk().withdraw()
image_file = askopenfilename(title='Load image file')
img = io.imread(image_file)
print('Image File:', image_file)
print("Image Shape:", img.shape)

h, w, d = img.shape

# Extract layers from the multilayer tiff file and do some adjustments
if d == 5:
    layer_RGB = img[:, :, 0:3]
#     layer_IR = img[:, :, 3]
    layer_mask = img[:, :, 4]
    # Change from 16 bit value to 0~1
    layer_RGB = layer_RGB / 65535
    # Set background to black
    layer_RGB[np.where(layer_mask == 0)] = 0
#     layer_IR[np.where(layer_mask == 0)] = 0
    # Set image to show
    img2show = layer_RGB
elif d == 4:
    layer_RGB = img[:, :, 0:3]
    layer_mask = img[:, :, 3]
    # Change from 16 bit value to 0~1
#     layer_RGB = layer_RGB / 65535
    # Set background to black
    layer_RGB[np.where(layer_mask == 0)] = 0
    # Set image to show
    img2show = layer_RGB
elif d == 2:
    layer_IR = img[:, :, 0]
    layer_mask = img[:, :, 1]
    # Set background to black
    layer_IR[np.where(layer_mask == 0)] = 0
    # Set image to show
    img2show = layer_IR

# Play sound when done
def done():
    print('\nDone')
    
    if platform.system() == 'Windows':
        import winsound
        winsound.Beep(500, 1000)
    elif platform.system() == 'Darwin':
        import os
        os.system( "say Done" )
    elif platform == 'linux' or platform == 'linux2':
        import os
        os.system( "say Done" )
        
    # Reduce use of memory
    gc.collect()

done()

In [ ]:
# Interested area pre-selection - Crop out the image for faster process

fig, ax = plt.subplots(figsize=(7, 7))
plt.subplots_adjust(left=0.05, bottom=0.05, right=0.95, top=0.95)
myax = ax.imshow(img2show)

def line_select_callback(eclick, erelease):
    x1, y1 = eclick.xdata, eclick.ydata
    x2, y2 = erelease.xdata, erelease.ydata
    
# def toggle_rect_selector(event):
#     if event.key in ['X', 'x'] and toggle_rect_selector.RS.active:
#         toggle_rect_selector.RS.set_active(False)
#         print('Selector deactivated')
#     if event.key in ['C', 'c'] and not toggle_rect_selector.RS.active:
#         toggle_rect_selector.RS.set_active(True)
#         print('Selector activated')

# toggle_rect_selector.RS = RectangleSelector(ax, line_select_callback, drawtype='box', useblit=True, button=[1], minspanx=50, minspany=50,
#                                             rectprops=dict(facecolor='red', edgecolor='black', alpha=0.1, fill=True), 
#                                             spancoords='pixels', interactive=True)

rs = RectangleSelector(ax, line_select_callback, drawtype='box', useblit=True, button=[1], minspanx=50, minspany=50,
                  rectprops=dict(facecolor='red', edgecolor='black', alpha=0.1, fill=True), spancoords='pixels', interactive=True)


# plt.connect('key_press_event', toggle_rect_selector)
plt.show()

In [ ]:
# Record up-left & bottom-right corners of area of interest

selected_area_corners = rs.corners
ul_x, ul_y = np.round([selected_area_corners[0][0], selected_area_corners[1][1]])
br_x, br_y = np.round([selected_area_corners[0][2], selected_area_corners[1][2]])

ul_x = ul_x.astype(int)
ul_y = ul_y.astype(int)
br_x = br_x.astype(int)
br_y = br_y.astype(int)

done()

In [ ]:
# Set boundary for each plot

def onselect(vert):
    global one_plot_vertices
    one_plot_vertices = np.round(np.array(vert))

def load(b):
    print(b)
    info_box.value = 'Failed'
    
def save(b):
    global one_plot_vertices, ax
    if text_box.value == '':
        info_box.value = 'Failed! Please input plot name!'
        return
    elif text_box.value in plot_names:
        info_box.value = 'Failed! Plot already EXISTS!'
        return
    elif one_plot_vertices is None:
        info_box.value = 'Failed! Please select vertices!'
        return
    else:
        plot_names.append(text_box.value)
        plot_vertices.append(one_plot_vertices)
        polygon = patches.Polygon(one_plot_vertices, True, facecolor = matplotlib.colors.to_rgba('red', 0.1), edgecolor=matplotlib.colors.to_rgba('orange', 0.5))
        ax.add_patch(polygon)
        text_loc = np.mean(one_plot_vertices, 0)
        ax.text(text_loc[0], text_loc[1], text_box.value, ha='center', va='center')
        one_plot_vertices = None
        info_box.value = 'Success! Plot ' + text_box.value + ' vertices saved!'
        
def save_when_enter_pressed(event):
    if event.key == 'enter':
        save(text_box)
    fig.canvas.draw()
        
    

    
plot_names = []
plot_vertices = []
one_plot_vertices = None

button_layout = widgets.Layout(width='50%')
instruction_box = widgets.Textarea(value='1. Select vertices of plot \n2. Input plot name \n3. Save plot \n4. Select for next plot or end',
                                   placeholder='Instructions will be printed here', 
                                   disabled=True, layout=widgets.Layout(height='40%', width='50%'))
text_box = widgets.Text(placeholder='Input plot name here', layout=button_layout)
button_load = widgets.Button(description="Load plot name From File", layout=button_layout)
button_save = widgets.Button(description="Save Plot (Press Enter)", layout=button_layout)
info_box = widgets.Text(placeholder='ERROR/Info will be printed in here', layout=button_layout, disabled=True)

box_layout = widgets.Layout(align_items='center', width='50%')
buttons = widgets.VBox(children=[instruction_box, text_box, button_load, button_save, info_box], layout=box_layout)
out = widgets.Output()
all_widgets = widgets.HBox(children=[out, buttons])
display(all_widgets)
button_load.on_click(load)
button_save.on_click(save)
text_box.on_submit(save)



with out:
    fig, ax = plt.subplots(figsize=(5, 5))
    fig.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9)
    ax.imshow(img2show[ul_y:br_y, ul_x:br_x])
    ps = PolygonSelector(ax, onselect, useblit=True)
    fig.canvas.mpl_connect('key_press_event', save_when_enter_pressed)
    plt.show()

In [ ]:
# Save GPS coordinates to file

ds = gdal.Open(image_file)
gt = ds.GetGeoTransform()

plot_vertices_gps = []
for i in range(len(plot_names)):
    one_plot_vertices = plot_vertices[i]
    one_plot_vertices_gps = []
    for vertex in one_plot_vertices:
        x = ul_x + vertex[0]
        y = ul_y + vertex[1]
        
        xgeo = gt[0] + x*gt[1] + y*gt[2]
        ygeo = gt[3] + x*gt[4] + y*gt[5]
        one_plot_vertices_gps.append([xgeo, ygeo])
    one_plot_vertices_gps = np.array(one_plot_vertices_gps)
    plot_vertices_gps.append(one_plot_vertices_gps)


def ask_to_save():
    fn = image_file.split('/')[-2] + '_' + image_file.split('/')[-1].split('.')[0]
    file_name = asksaveasfilename(filetypes=[('pickle', '*.pkl')], title='Save plot locations', initialfile=fn+'_plot_loc')
    if not file_name:
        return
    if not file_name.endswith('.pkl'):
        file_name += '.pkl'
        
    try:
        with open(file_name, 'wb') as f:
            pickle.dump(plot_names, f)
            pickle.dump(plot_vertices_gps, f)
        print('GPS coordinates saved to', file_name)
    except Exception as e:
        showerror(type(e).__name__, str(e))


Tk().withdraw()
ask_to_save()

done()